Użyj każdą z tych funkcji 
* `unix_timestamp()` 
* `date_format()`
* `to_unix_timestamp()`
* `from_unixtime()`
* `to_date()` 
* `to_timestamp()` 
* `from_utc_timestamp()` 
* `to_utc_timestamp()`

In [0]:
from pyspark.sql.functions import *

kolumny = ["timestamp", "unix", "Date"]
dane = [("2015-03-22T14:13:34", 1646641525847, "May, 2021"),
        ("2015-03-22T15:03:18", 1646641557555, "Mar, 2021"),
        ("2015-03-22T14:38:39", 1646641578622, "Jan, 2021")]

dataFrame = spark.createDataFrame(dane, kolumny). \
    withColumn("unix_timestamp", unix_timestamp()). \
    withColumn("date_format", date_format("timestamp", "dd/MM/yyyy")). \
    withColumn("to_unix_timestamp", unix_timestamp(current_date(), "yyyy-MM-dd")). \
    withColumn("from_unixtime", from_unixtime("unix_timestamp")). \
    withColumn("to_date", to_date("timestamp")). \
    withColumn("to_timestamp", to_timestamp("from_unixtime")). \
    withColumn("from_utc_timestamp", from_utc_timestamp("timestamp", tz="UTC")). \
    withColumn("to_utc_timestamp", to_utc_timestamp("from_unixtime", tz="UTC"))

display(dataFrame)

In [0]:
dataFrame.printSchema()

## unix_timestamp(..) & cast(..)

Konwersja **string** to a **timestamp**.

Lokalizacja funkcji 
* `pyspark.sql.functions` in the case of Python
* `org.apache.spark.sql.functions` in the case of Scala & Java

## 1. Zmiana formatu wartości timestamp yyyy-MM-dd'T'HH:mm:ss 
`unix_timestamp(..)`

Dokumentacja API `unix_timestamp(..)`:
> Convert time string with given pattern (see <a href="http://docs.oracle.com/javase/tutorial/i18n/format/simpleDateFormat.html" target="_blank">SimpleDateFormat</a>) to Unix time stamp (in seconds), return null if fail.

`SimpleDataFormat` is part of the Java API and provides support for parsing and formatting date and time values.

In [0]:
df2 = dataFrame.withColumn("timestamp", unix_timestamp(current_timestamp(), "yyyy-MM-dd'T'HH:mm:ss"))
display(df2)

2. Zmień format zgodnie z klasą `SimpleDateFormat`**yyyy-MM-dd HH:mm:ss**
  * a. Wyświetl schemat i dane żeby sprawdzicz czy wartości się zmieniły

In [0]:
from pyspark.sql.functions import unix_timestamp, from_unixtime

df2.printSchema()
zmianaFormatu = df2.withColumn("timestamp", df2["timestamp"].cast(StringType()))
zmianaFormatu.printSchema()

In [0]:
#unix_timestamp
display(zmianaFormatu)

## Stwórz nowe kolumny do DataFrame z wartościami year(..), month(..), dayofyear(..)

In [0]:
#date_format
yearDate = df2.withColumn("year", year(to_date("date_format", "dd/MM/yyyy")))
display(yearDate)

In [0]:
#to_date()
toDate = df2.withColumn("month", month(to_date("to_date", "dd/MM/yyyy")))
display(toDate)

In [0]:
#from_unixtime()
fromUnix = df2.withColumn("day", dayofmonth("from_unixtime"))
display(fromUnix)

In [0]:
#to_timestamp()
toTimestamp = df2.withColumn("dayofyear", dayofyear("from_unixtime"))
display(toTimestamp)


In [0]:
#to_utc_timestamp()
toUtcTimestamp = df2.withColumn("minute", minute("from_unixtime"))
display(toUtcTimestamp)



In [0]:
#from_utc_timestamp()
fromUtcTimestamp = df2.withColumn("second", second("from_unixtime"))
display(fromUtcTimestamp)

In [0]:
%fs ls dbfs:/databricks-datasets/flights/

path,name,size,modificationTime
dbfs:/databricks-datasets/flights/README.md,README.md,412,1457766852000
dbfs:/databricks-datasets/flights/airport-codes-na.txt,airport-codes-na.txt,11411,1457749605000
dbfs:/databricks-datasets/flights/departuredelays.csv,departuredelays.csv,33396236,1457749605000


In [0]:
df = spark.read.format("csv").option("header","true").option("inferSchema","true").load("dbfs:/databricks-datasets/flights/departuredelays.csv/")
display(df)
df.printSchema()

root
 |-- date: integer (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

schema = StructType([
    StructField("date",IntegerType(),False),
    StructField("delay",IntegerType(),False),
    StructField("distance",IntegerType(),False),
    StructField("origin",StringType(),False),
    StructField("destination",StringType(),False),
])

schema

Out[5]: StructType([StructField('date', IntegerType(), False), StructField('delay', IntegerType(), False), StructField('distance', IntegerType(), False), StructField('origin', StringType(), False), StructField('destination', StringType(), False)])

In [0]:
df = spark.read.format("csv").option("header","true").schema(schema).load("dbfs:/databricks-datasets/flights/departuredelays.csv/")

display(df)
df.printSchema()

root
 |-- date: integer (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



In [0]:
df_permissive = spark.read.format("csv").schema(schema).options(sep=";",header=True, mode="PERMISSIVE").load("dbfs:/databricks-datasets/flights/departuredelays.csv/")
df_dropmal = spark.read.format("csv").schema(schema).options(sep=";",header=True, mode="DROPMALFORMED").load("dbfs:/databricks-datasets/flights/departuredelays.csv/")
df_failfast = spark.read.format("csv").schema(schema).options(sep=";",header=True, mode="FAILFAST").load("dbfs:/databricks-datasets/flights/departuredelays.csv/")
df_badrecords = spark.read.format("csv").schema(schema).options(sep=";",header=True).option("badRecordsPath","/mnt/source/badrecords").load("dbfs:/databricks-datasets/flights/departuredelays.csv/")

display(df_permissive)
display(df_dropmal)
display(df_failfast)
display(df_badrecords)

date,delay,distance,origin,destination
null,null,null,null,null
null,null,null,null,null
null,null,null,null,null
null,null,null,null,null
null,null,null,null,null
null,null,null,null,null
null,null,null,null,null
null,null,null,null,null
null,null,null,null,null
null,null,null,null,null


date,delay,distance,origin,destination


org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 15.0 failed 1 times, most recent failure: Lost task 0.0 in stage 15.0 (TID 41) (ip-10-172-176-207.us-west-2.compute.internal executor driver): com.databricks.sql.io.FileReadException: Error while reading file dbfs:/databricks-datasets/flights/departuredelays.csv.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logFileNameAndThrow(FileScanRDD.scala:704)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:673)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:796)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.$anonfun$hasNext$1(FileScanRDD.scala:496)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:486)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:82)
	at org.apache.spark.sql.execution.collect.Collector.$anonfun$processFunc$1(Collector.scala:208)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$3(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$1(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:55)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:179)
	at org.apache.spark.scheduler.Task.$anonfun$run$5(Task.scala:142)
	at com.databricks.unity.EmptyHandle$.runWithAndClose(UCSHandle.scala:126)
	at org.apache.spark.scheduler.Task.$anonfun$run$1(Task.scala:142)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.Task.run(Task.scala:97)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$13(Executor.scala:904)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1741)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:907)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:761)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)
Caused by: org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInRecordParsingError(QueryExecutionErrors.scala:1936)
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:103)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:507)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at org.apache.spark.util.CompletionIterator.hasNext(CompletionIterator.scala:31)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:619)
	... 29 more
Caused by: org.apache.spark.sql.catalyst.util.BadRecordException: org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Mal

In [0]:
df.write.options(header=True, sep=',').format("parquet").mode("overwrite").save("dbfs:/tmp/yeyyy.parquet")

df_test = spark.read.format("parquet").schema(schema).options(header=True).load("dbfs:/tmp/yeyyy.parquet")
display(df_test)

date,delay,distance,origin,destination
1021738,54,599,IAH,ATL
1021708,169,793,IAH,CLT
1022009,57,175,IAH,LFT
1022058,7,175,IAH,CRP
1020738,-5,898,IAH,MSP
1021421,73,906,IAH,RDU
1020800,0,571,IAH,BNA
1021908,4,373,IAH,TUL
1020817,-5,489,IAH,BHM
1022058,118,268,IAH,BRO
